<a href="https://colab.research.google.com/github/ghostwalkin/Multimodal-finetuning-ImgText-pair/blob/main/Finetune_CLIP_Amazon_products_Data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#dependencies



In [1]:
!pip install condacolab

In [2]:
!pip install transformers datasets requests sentence-transformers huggingface-hub  --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 51.8 MB/s eta 0:00:00


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
from google.colab import files
files.upload() #upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dparijat2","key":"66ae7bf19907ff638fff2cab4e597634"}'}

In [6]:
#kaggle auth
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#data download



In [7]:
!kaggle datasets download asaniczka/amazon-products-dataset-2023-1-4m-products

Dataset URL: https://www.kaggle.com/datasets/asaniczka/amazon-products-dataset-2023-1-4m-products
License(s): ODC Attribution License (ODC-By)


In [8]:
from zipfile import ZipFile
with ZipFile('amazon-products-dataset-2023-1-4m-products.zip', 'r') as zipObj:
   zipObj.extractall()

In [9]:
!rm -rf amazon-products-dataset-2023-1-4m-products.zip

#preprocess

Have to preprocess the data in anchor, positive and negative colums; hence going to use the image column as anchor, real title as positive and some random title from different category as negative samples.

In [10]:
import pandas as pd
df = pd.read_csv('amazon_products.csv')
df=df[['title','imgUrl',"category_id"]]


In [11]:
sampled_df=df.sample(n=50000,random_state=202)

##positive and negative sampling

In [12]:
#pd-to dataset
from datasets import Dataset,load_dataset,DatasetDict

dataset=Dataset.from_pandas(sampled_df)

In [13]:
dataset

Dataset({
    features: ['title', 'imgUrl', 'category_id', '__index_level_0__'],
    num_rows: 50000
})

In [17]:
#@title hard-negative mining
from sentence_transformers.util import mine_hard_negatives
from sentence_transformers import SentenceTransformer


model=SentenceTransformer("all-mpnet-base-v2")

hardn_dataset=mine_hard_negatives(dataset=dataset,
    model=model,
    anchor_column_name="imgUrl",
    positive_column_name ="title",
    range_min=5,
    range_max=10,
    num_negatives=3,
    sampling_strategy="top",
    batch_size=150,
    use_faiss=True,)

Found 49755 unique queries out of 50000 total queries.
Found an average of 1.005 positives per query.


Batches:   0%|          | 0/333 [00:00<?, ?it/s]

Batches:   0%|          | 0/332 [00:00<?, ?it/s]

Querying FAISS index: 100%|██████████| 4/4 [00:45<00:00, 11.39s/it]


Metric       Positive       Negative     Difference
Count          50,000        150,000               
Mean           0.1315         0.3580        -0.2266
Median         0.1320         0.3563        -0.2256
Std            0.0612         0.0126         0.0615
Min           -0.1056         0.2683        -0.5280
25%            0.0898         0.3504        -0.2683
50%            0.1320         0.3563        -0.2256
75%            0.1731         0.3635        -0.1846
Max            0.3845         0.5758         0.0139


In [18]:
triplet_df=hardn_dataset.to_pandas()
triplet_df.head()

,imgUrl,title,negative
0,https://m.media-amazon.com/images/I/511xeVXvaE...,Gold Earrings Dangle Drop Earrings For Women F...,Head Case Designs Officially Licensed Assassin...
1,https://m.media-amazon.com/images/I/511xeVXvaE...,Gold Earrings Dangle Drop Earrings For Women F...,Head Case Designs Officially Licensed Animal C...
2,https://m.media-amazon.com/images/I/511xeVXvaE...,Gold Earrings Dangle Drop Earrings For Women F...,Head Case Designs Officially Licensed Assassin...
3,https://m.media-amazon.com/images/I/5117LAB-wb...,Cute Night Light for Kids - Nightlight for Chi...,Stupell Industries Game On Bold Gamer Phrase R...
4,https://m.media-amazon.com/images/I/5117LAB-wb...,Cute Night Light for Kids - Nightlight for Chi...,Head Case Designs Officially Licensed Assassin...


In [ ]:
data=triplet_df.copy()
train_frac=0.6
val_frac=0.2
test_frac=0.2

train_size = int(train_frac * len(data))
valid_size = int(val_frac * len(data))

# create train, validation, and test datasets
df_train = data[:train_size]
df_valid = data[train_size:train_size + valid_size]
df_test = data[train_size + valid_size:]

In [ ]:
data_dict=DatasetDict()
data_dict['train']=Dataset.from_pandas(df_train)
data_dict['valid']=Dataset.from_pandas(df_valid)
data_dict['test']=Dataset.from_pandas(df_test)

In [ ]:
data_dict["train"].shape[0]+data_dict["valid"].shape[0]+data_dict["test"].shape[0]

250000

In [ ]:
ds=data_dict

# data cleaning and preprocessing


In [ ]:
import requests
from PIL import Image,UnidentifiedImageError
import concurrent.futures
from tqdm import tqdm

def chk_img(img):
  try:
    response = requests.get(img, stream=True, timeout=10) #added timeout.
    response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
    Image.open(response.raw).verify() #verify the image is not truncated.
    return None
  except (requests.exceptions.RequestException, OSError, UnidentifiedImageError) as e:
        return img

image_list=ds["train"]["imgUrl"]+ds["test"]["imgUrl"]+ds["valid"]["imgUrl"]
image_list=list(set(image_list))

with concurrent.futures.ThreadPoolExecutor() as executor:
    corrupt_image = list(tqdm(executor.map(chk_img, image_list), total=len(image_list), desc="Checking Images"))

corrupt_image=list(filter(None, corrupt_image))
print(corrupt_image)


In [ ]:
ds["train"]=ds["train"].filter(lambda x:x["imgUrl"] not in corrupt_image)
ds["test"]=ds["test"].filter(lambda x:x["imgUrl"] not in corrupt_image)
ds["valid"]=ds["valid"].filter(lambda x:x["imgUrl"] not in corrupt_image)
ds

In [ ]:
def preprocess(batch):
  image_list=[Image.open(requests.get(url,stream=True).raw) for url in batch["imgUrl"]]
  batch["title"]=[x[:77] for x in batch["title"]]
  batch["negative"]=[x[:77] for x in batch["negative"]]

  return {
        "anchor": image_list,
        "positive": batch["title"],
        "negative": batch["negative"]
    }

columns_to_remove=["imgUrl","title"]

ds=ds.map(preprocess,remove_columns=columns_to_remove,batched=True)
ds["train"]=ds["train"].select_columns(["anchor","positive","negative"])
ds["test"]=ds["test"].select_columns(["anchor","positive","negative"])
ds["valid"]=ds["valid"].select_columns(["anchor","positive","negative"])

ds

# publish on hub

In [ ]:
ds.push_to_hub("test_repo/triplet_data")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/150 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/50 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/50 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/dparijat/amazon-image-title-triplet-250k/commit/7fa6988387b1ebca93d33b33a4949d4aa44e755d', commit_message='Upload dataset', commit_description='', oid='7fa6988387b1ebca93d33b33a4949d4aa44e755d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/dparijat/amazon-image-title-triplet-250k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='dparijat/amazon-image-title-triplet-250k'), pr_revision=None, pr_num=None)